In [ ]:
import requests
import re
import time
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
test = [224517, 161936]
print(str(test)[1:-1])

224517, 161936


In [ ]:
url = 'https://boardgamegeek.com/xmlapi2/thing'
parameters = {'id':'224517,161936'}

resp = requests.get(url, params=parameters, timeout=20)
resp
resp.close()

In [ ]:
tree = ET.fromstring(resp.text)
tree

<Element 'items' at 0x7f75932bfc70>

In [ ]:
# Get game categories, mechanics, and families
categories = []
mechanics = []
families = []

for x in tree[0].findall('link'):
  y = x.attrib
  if(y['type'] == 'boardgamecategory'):
    categories.append(y['value'])
  elif(y['type'] == 'boardgamemechanic'):
    mechanics.append(y['value'])
  elif(y['type'] == 'boardgamefamily'):
    families.append(y['value'])

print(categories)
print(mechanics)
print(families)


['Economic', 'Industry / Manufacturing', 'Post-Napoleonic', 'Trains', 'Transportation']
['Hand Management', 'Income', 'Loans', 'Market', 'Network and Route Building', 'Tags', 'Tech Trees / Tech Tracks', 'Turn Order: Stat-Based', 'Variable Set-up']
['Cities: Birmingham (England)', 'Components: Multi-Use Cards', 'Country: England', 'Crowdfunding: Kickstarter', 'Crowdfunding: Spieleschmiede', 'Digital Implementations: Steam', 'Digital Implementations: Tabletopia', 'Food & Drink: Beer', 'Game: Brass', 'History: Industrial Revolution', 'Misc: Watch It Played How To Videos', 'Organizations: The Game Artisans of Canada', 'Theme: Canals']


In [ ]:
# get long description
tree[0].find('description').text

"Brass: Birmingham is an economic strategy game sequel to Martin Wallace' 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.&#10;&#10;As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands.&#10;&#10;Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):&#10;&#10;1) Build - Pay required resources and place an industry tile.&#10;2) Network - Add a rail / canal link, expanding your network.&#10;3) Develop - Increase the VP value of an industry.&#10;4) Sell - Sell your cotton, manufactured goods and pottery.&#10;5) Loan - Take a &pound;30 loan and reduce your income.&#10;&#10;Brass: Birmingham also features a new sixth action:&#10;&#10;6) Scout - Discard three cards and take a wild location

In [ ]:
# thumbnail and main images
print(tree[0].find('thumbnail').text)
print(tree[0].find('image').text)

https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__thumb/img/o18rjEemoWaVru9Y2TyPwuIaRfE=/fit-in/200x150/filters:strip_icc()/pic3490053.jpg
https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__original/img/FpyxH41Y6_ROoePAilPNEhXnzO8=/0x0/filters:format(jpeg)/pic3490053.jpg


In [ ]:
for x in tree[0].findall('name'):
  y = x.attrib
  if y['type'] == 'primary':
    print(y['value'])

Brass: Birmingham


In [ ]:
# basic int attributes
print(int(tree[0].find('minplayers').attrib['value']))
print(int(tree[0].find('maxplayers').attrib['value']))
print(int(tree[0].find('playingtime').attrib['value']))
print(int(tree[0].find('minplaytime').attrib['value']))
print(int(tree[0].find('maxplaytime').attrib['value']))

2
4
120
60
120


In [ ]:
tree[0].find('yearpublished').attrib['value']

'2018'

In [ ]:
tree[0].get('id')

'224517'

In [ ]:
def parse_game_info(game):

  game_dict = {}

  # Get BGG database ID
  game_dict['Game_ID'] = int(game.get('id'))

  # Get primary game name
  for x in game.findall('name'):
    y = x.attrib
    if y['type'] == 'primary':
      game_dict['Game Name'] = y['value']

  # Get year published
  game_dict['Year Published'] = game.find('yearpublished').attrib['value']

  # Get thumbnail and image
  try:
    game_dict['Thumbnail'] = game.find('thumbnail').text
  except:
    game_dict['Thumbnail'] = ''
    
  try:
    game_dict['Image'] = game.find('image').text
  except:
    game_dict['Image'] = ''

  # Get game categories, mechanics, families
  # and designers
  categories = []
  mechanics = []
  families = []
  designers = []

  for x in game.findall('link'):
    y = x.attrib
    link_type = y['type']
    link_value = y['value']

    if(link_type == 'boardgamecategory'):
      categories.append(link_value)
    elif(link_type == 'boardgamemechanic'):
      mechanics.append(link_value)
    elif(link_type == 'boardgamefamily'):
      families.append(link_value)
    elif(link_type == 'boardgamedesigner'):
      designers.append(link_value)

  game_dict['Categories'] = str(categories)
  game_dict['Mechanics'] = str(mechanics)
  game_dict['Families'] = str(families)
  game_dict['Designers'] = str(designers)

  # Get the detailed description
  try:
    game_dict['Long_Description'] = game.find('description').text
  except:
    print('Unable to get long description {}.'.format(game_dict['Game_ID']))
    game_dict['Long_Description'] = ''

  # get player number and play time information
  game_dict['Minplayers'] = int(game.find('minplayers').attrib['value'])
  game_dict['Maxplayers'] = int(game.find('maxplayers').attrib['value'])
  game_dict['Playingtime'] = int(game.find('playingtime').attrib['value'])
  game_dict['Minplaytime'] = int(game.find('minplaytime').attrib['value'])
  game_dict['Maxplaytime'] = int(game.find('maxplaytime').attrib['value'])

  return game_dict

In [ ]:
parse_game_info(tree[0])

{'Game_ID': 224517,
 'Game Name': 'Brass: Birmingham',
 'Year Published': '2018',
 'Thumbnail': 'https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__thumb/img/o18rjEemoWaVru9Y2TyPwuIaRfE=/fit-in/200x150/filters:strip_icc()/pic3490053.jpg',
 'Image': 'https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__original/img/FpyxH41Y6_ROoePAilPNEhXnzO8=/0x0/filters:format(jpeg)/pic3490053.jpg',
 'Categories': "['Economic', 'Industry / Manufacturing', 'Post-Napoleonic', 'Trains', 'Transportation']",
 'Mechanics': "['Hand Management', 'Income', 'Loans', 'Market', 'Network and Route Building', 'Tags', 'Tech Trees / Tech Tracks', 'Turn Order: Stat-Based', 'Variable Set-up']",
 'Families': "['Cities: Birmingham (England)', 'Components: Multi-Use Cards', 'Country: England', 'Crowdfunding: Kickstarter', 'Crowdfunding: Spieleschmiede', 'Digital Implementations: Steam', 'Digital Implementations: Tabletopia', 'Food & Drink: Beer', 'Game: Brass', 'History: Industrial Revolution', 'Misc: Watch It Played Ho

In [ ]:
parse_game_info(tree[1])

{'Game_ID': 161936,
 'Game Name': 'Pandemic Legacy: Season 1',
 'Year Published': '2015',
 'Thumbnail': 'https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__thumb/img/NQQcjS31TO0DE246N9rpt0hd9eo=/fit-in/200x150/filters:strip_icc()/pic2452831.png',
 'Image': 'https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__original/img/PlzAH7swN1nsFxOXbfUvE3TkE5w=/0x0/filters:format(png)/pic2452831.png',
 'Categories': "['Environmental', 'Medical']",
 'Mechanics': "['Action Points', 'Cooperative Game', 'Hand Management', 'Legacy Game', 'Point to Point Movement', 'Scenario / Mission / Campaign Game', 'Set Collection', 'Tags', 'Trading', 'Variable Player Powers']",
 'Families': "['Components: Map (Global Scale)', 'Components: Multi-Use Cards', 'Game: Pandemic', 'Mechanism: Campaign Games', 'Mechanism: Legacy', 'Medical: Diseases', 'Misc: Limited Replayability', 'Misc: Made by Panda', 'Occupation: Dispatcher', 'Occupation: Medic / Doctor / Nurses', 'Occupation: Researcher / Scientist', 'Region: The W

In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
filepath = Path('/content/gdrive/MyDrive/DataWorks MD Talk/games.csv')
df = pd.read_csv(filepath)
df.head()

,Game_Name,Game_ID,Rank,Geek_Rating,Average_Rating,Number_of_Voters,Description
0,Brass: Birmingham,224517,1,8.426,8.62,37388,"Build networks, grow industries, and navigate ..."
1,Pandemic Legacy: Season 1,161936,2,8.395,8.54,50114,Mutating diseases are spreading around the wor...
2,Gloomhaven,174430,3,8.394,8.63,57589,Vanquish monsters with strategic cardplay. Ful...
3,Ark Nova,342942,4,8.295,8.54,28630,"Plan and build a modern, scientifically manage..."
4,Twilight Imperium: Fourth Edition,233078,5,8.241,8.62,20463,"Build an intergalactic empire through trade, r..."


In [ ]:
url = 'https://boardgamegeek.com/xmlapi2/thing'
parameters = {'id':''}

resp = requests.get(url, params=parameters, timeout=20)
resp
resp.close()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Game_Name         10000 non-null  object 
 1   Game_ID           10000 non-null  int64  
 2   Rank              10000 non-null  int64  
 3   Geek_Rating       10000 non-null  float64
 4   Average_Rating    10000 non-null  float64
 5   Number_of_Voters  10000 non-null  int64  
 6   Description       6954 non-null   object 
dtypes: float64(2), int64(3), object(2)
memory usage: 547.0+ KB


In [ ]:
ids = []
game_info = pd.DataFrame()
tries = 5
errors = []

for id in df['Game_ID']:
  ids.append(id)

  # Query for games in groups of 100
  if len(ids) == 100:

    # Make a compact string of game ids
    ids_str = str(ids)[1:-1].replace(' ', '')
    parameters['id'] = ids_str
    resp = None
    
    # Make requests until we can get a response
    for i in range(tries):
      try:
        resp = requests.get(url, params=parameters, timeout=20)
        break
      except Exception as e:
        print('Request failed trying again in 5s.')
        print(e)
        time.sleep(5)
    else:
      ValueError('BGG Unresponsive')

    #Format the response for parsing    
    tree = ET.fromstring(resp.text)
    resp.close()

    # parse game info and add to dataframe
    for game in tree:
      try:
        parsed_game = pd.DataFrame([parse_game_info(game)])
        game_info = pd.concat([game_info, parsed_game]) 
      except:
        print('Error parsing. Moving to the next.')
        errors.append(game)
    
    ids = []
    print('Total rows: ', len(game_info.index))
    time.sleep(2)

print('Done')

Total rows:  100
Total rows:  200
Total rows:  300
Total rows:  400
Total rows:  500
Total rows:  600
Total rows:  700
Total rows:  800
Total rows:  900
Total rows:  1000
Total rows:  1100
Total rows:  1200
Total rows:  1300
Total rows:  1400
Total rows:  1500
Total rows:  1600
Total rows:  1700
Total rows:  1800
Total rows:  1900
Total rows:  2000
Total rows:  2100
Total rows:  2200
Total rows:  2300
Request failed trying again in 5s.
HTTPSConnectionPool(host='boardgamegeek.com', port=443): Read timed out. (read timeout=20)
Total rows:  2400
Total rows:  2500
Total rows:  2600
Total rows:  2700
Total rows:  2800
Total rows:  2900
Total rows:  3000
Total rows:  3100
Total rows:  3200
Total rows:  3300
Total rows:  3400
Total rows:  3500
Total rows:  3600
Total rows:  3700
Total rows:  3800
Total rows:  3900
Total rows:  4000
Total rows:  4100
Total rows:  4200
Total rows:  4300
Total rows:  4400
Total rows:  4500
Total rows:  4600
Total rows:  4700
Total rows:  4800
Total rows:  4900
T

In [ ]:
game_info

,Game_ID,Game Name,Year Published,Thumbnail,Image,Categories,Mechanics,Families,Designers,Long_Description,Minplayers,Maxplayers,Playingtime,Minplaytime,Maxplaytime
0,224517,Brass: Birmingham,2018,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,"['Economic', 'Industry / Manufacturing', 'Post...","['Hand Management', 'Income', 'Loans', 'Market...","['Cities: Birmingham (England)', 'Components: ...","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']",Brass: Birmingham is an economic strategy game...,2,4,120,60,120
0,161936,Pandemic Legacy: Season 1,2015,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Components: Map (Global Scale)', 'Components...","['Rob Daviau', 'Matt Leacock']",Pandemic Legacy is a co-operative campaign gam...,2,4,60,60,60
0,174430,Gloomhaven,2017,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Action Queue', 'Action Retrieval', 'Campaign...","['Category: Dungeon Crawler', 'Components: Min...",['Isaac Childres'],Gloomhaven is a game of Euro-inspired tactica...,1,4,120,60,120
0,342942,Ark Nova,2021,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,"['Animals', 'Economic', 'Environmental']","['End Game Bonuses', 'Hand Management', 'Hexag...","['Components: Hexagonal Tiles', 'Digital Imple...",['Mathias Wigge'],"In Ark Nova, you will plan and design a modern...",1,4,150,90,150
0,233078,Twilight Imperium: Fourth Edition,2017,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,"['Civilization', 'Economic', 'Exploration', 'N...","['Action Drafting', 'Area Majority / Influence...","['Components: Hexagonal Tiles', 'Components: M...","['Dane Beltrami', 'Corey Konieczka', 'Christia...",Twilight Imperium (Fourth Edition) is a game o...,3,6,480,240,480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,221367,Cartoon Network Crossover Crisis: Animation An...,2017,https://cf.geekdo-images.com/S4ml16Y9vnqnecZMt...,https://cf.geekdo-images.com/S4ml16Y9vnqnecZMt...,"['Card Game', 'Humor', 'Movies / TV / Radio th...","['Deck, Bag, and Pool Building']","['Brands: Television (TV) Networks', 'Game: Ce...","['Matt Dunn', 'Matt Hyra']",The calamity continues with the latest Cartoon...,2,4,45,30,45
0,96061,"Jours de Gloire Campagne IV: Allemagne 1813, d...",2011,https://cf.geekdo-images.com/XW2epRAfXvSrpkWQa...,https://cf.geekdo-images.com/XW2epRAfXvSrpkWQa...,"['Napoleonic', 'Wargame']","['Action Points', 'Point to Point Movement']","['Cities: Leipzig (Saxony, Germany)', 'Series:...",['Frédéric Bey'],Jours de Gloire Campagne is a game series desi...,2,5,120,120,120
0,356224,Battles of Normandy: A Solitaire Wargame,2022,https://cf.geekdo-images.com/HAxMm1XBy4DKoMDVm...,https://cf.geekdo-images.com/HAxMm1XBy4DKoMDVm...,"['Book', 'Wargame', 'World War II']","['Action/Event', 'Dice Rolling', 'Grid Movemen...","['Country: France', 'Country: USA', 'Players: ...",['Mike Lambo'],As part of the D-Day landings in Normandy in 1...,1,1,60,30,60
0,118215,Würfelwurst,2012,https://cf.geekdo-images.com/CU1bBXTpmacj7Is2a...,https://cf.geekdo-images.com/CU1bBXTpmacj7Is2a...,"['Animals', 'Dice']","['Dice Rolling', 'Re-rolling and Locking', 'Se...","['Animals: Birds', 'Animals: Insects', 'Animal...","['Markus Brand', 'Inka Brand']",W&uuml;rfelwurst (aka &quot;Dice-Sausage&quot;...,2,4,15,15,15


In [ ]:
try:
  parse_game_info(errors[0])
except:
  print('No errors')

No errors


In [ ]:
# parsed_game = pd.DataFrame([parse_game_info(errors[0])])
# game_info = pd.concat([game_info, parsed_game])

In [ ]:
game_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 0
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Game_ID           10000 non-null  int64 
 1   Game Name         10000 non-null  object
 2   Year Published    10000 non-null  object
 3   Thumbnail         10000 non-null  object
 4   Image             10000 non-null  object
 5   Categories        10000 non-null  object
 6   Mechanics         10000 non-null  object
 7   Families          10000 non-null  object
 8   Designers         10000 non-null  object
 9   Long_Description  10000 non-null  object
 10  Minplayers        10000 non-null  int64 
 11  Maxplayers        10000 non-null  int64 
 12  Playingtime       10000 non-null  int64 
 13  Minplaytime       10000 non-null  int64 
 14  Maxplaytime       10000 non-null  int64 
dtypes: int64(6), object(9)
memory usage: 1.2+ MB


In [ ]:
filepath = Path('/content/gdrive/MyDrive/DataWorks MD Talk/games_info.csv')
game_info.to_csv(filepath, index=False)  

In [ ]:
# Merged scrapped data with API data
games_full = df.merge(game_info, on='Game_ID', suffixes=('', '_'))
games_full.head()

,Game_Name,Game_ID,Rank,Geek_Rating,Average_Rating,Number_of_Voters,Description,Game Name,Year Published,Thumbnail,...,Categories,Mechanics,Families,Designers,Long_Description,Minplayers,Maxplayers,Playingtime,Minplaytime,Maxplaytime
0,Brass: Birmingham,224517,1,8.426,8.62,37388,"Build networks, grow industries, and navigate ...",Brass: Birmingham,2018,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg...,...,"['Economic', 'Industry / Manufacturing', 'Post...","['Hand Management', 'Income', 'Loans', 'Market...","['Cities: Birmingham (England)', 'Components: ...","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']",Brass: Birmingham is an economic strategy game...,2,4,120,60,120
1,Pandemic Legacy: Season 1,161936,2,8.395,8.54,50114,Mutating diseases are spreading around the wor...,Pandemic Legacy: Season 1,2015,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6K...,...,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Components: Map (Global Scale)', 'Components...","['Rob Daviau', 'Matt Leacock']",Pandemic Legacy is a co-operative campaign gam...,2,4,60,60,60
2,Gloomhaven,174430,3,8.394,8.63,57589,Vanquish monsters with strategic cardplay. Ful...,Gloomhaven,2017,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZ...,...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Action Queue', 'Action Retrieval', 'Campaign...","['Category: Dungeon Crawler', 'Components: Min...",['Isaac Childres'],Gloomhaven is a game of Euro-inspired tactica...,1,4,120,60,120
3,Ark Nova,342942,4,8.295,8.54,28630,"Plan and build a modern, scientifically manage...",Ark Nova,2021,https://cf.geekdo-images.com/SoU8p28Sk1s8MSvoM...,...,"['Animals', 'Economic', 'Environmental']","['End Game Bonuses', 'Hand Management', 'Hexag...","['Components: Hexagonal Tiles', 'Digital Imple...",['Mathias Wigge'],"In Ark Nova, you will plan and design a modern...",1,4,150,90,150
4,Twilight Imperium: Fourth Edition,233078,5,8.241,8.62,20463,"Build an intergalactic empire through trade, r...",Twilight Imperium: Fourth Edition,2017,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE...,...,"['Civilization', 'Economic', 'Exploration', 'N...","['Action Drafting', 'Area Majority / Influence...","['Components: Hexagonal Tiles', 'Components: M...","['Dane Beltrami', 'Corey Konieczka', 'Christia...",Twilight Imperium (Fourth Edition) is a game o...,3,6,480,240,480


In [ ]:
games_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Game_Name         10000 non-null  object 
 1   Game_ID           10000 non-null  int64  
 2   Rank              10000 non-null  int64  
 3   Geek_Rating       10000 non-null  float64
 4   Average_Rating    10000 non-null  float64
 5   Number_of_Voters  10000 non-null  int64  
 6   Description       6954 non-null   object 
 7   Game Name         10000 non-null  object 
 8   Year Published    10000 non-null  object 
 9   Thumbnail         10000 non-null  object 
 10  Image             10000 non-null  object 
 11  Categories        10000 non-null  object 
 12  Mechanics         10000 non-null  object 
 13  Families          10000 non-null  object 
 14  Designers         10000 non-null  object 
 15  Long_Description  10000 non-null  object 
 16  Minplayers        10000 non-null  int64  

In [ ]:
# Fill missing desriptions 
games_full['Description'].fillna('', inplace=True)
games_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Game_Name         10000 non-null  object 
 1   Game_ID           10000 non-null  int64  
 2   Rank              10000 non-null  int64  
 3   Geek_Rating       10000 non-null  float64
 4   Average_Rating    10000 non-null  float64
 5   Number_of_Voters  10000 non-null  int64  
 6   Description       10000 non-null  object 
 7   Game Name         10000 non-null  object 
 8   Year Published    10000 non-null  object 
 9   Thumbnail         10000 non-null  object 
 10  Image             10000 non-null  object 
 11  Categories        10000 non-null  object 
 12  Mechanics         10000 non-null  object 
 13  Families          10000 non-null  object 
 14  Designers         10000 non-null  object 
 15  Long_Description  10000 non-null  object 
 16  Minplayers        10000 non-null  int64  

In [ ]:
# Drop superfluous name column
games_full.drop(columns=['Game Name'], inplace=True)
games_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Game_Name         10000 non-null  object 
 1   Game_ID           10000 non-null  int64  
 2   Rank              10000 non-null  int64  
 3   Geek_Rating       10000 non-null  float64
 4   Average_Rating    10000 non-null  float64
 5   Number_of_Voters  10000 non-null  int64  
 6   Description       10000 non-null  object 
 7   Year Published    10000 non-null  object 
 8   Thumbnail         10000 non-null  object 
 9   Image             10000 non-null  object 
 10  Categories        10000 non-null  object 
 11  Mechanics         10000 non-null  object 
 12  Families          10000 non-null  object 
 13  Designers         10000 non-null  object 
 14  Long_Description  10000 non-null  object 
 15  Minplayers        10000 non-null  int64  
 16  Maxplayers        10000 non-null  int64  

In [ ]:
filepath = Path('/content/gdrive/MyDrive/DataWorks MD Talk/games_full.csv')
games_full.to_csv(filepath, index=False)  